# Get all Links

In [2]:
import os
import string
import requests
import json
import pandas as pd 
from bs4 import BeautifulSoup
from csv import writer

## Create links to every page

In [7]:
base_url = 'https://www.commonsensemedia.org/book-reviews'
page = '?page='
all_pages = range(1,291)
all_pages_list = [base_url+page+str(p) for p in all_pages]

## Create a folder to hold the scraped pages

In [17]:
!mkdir lexile/raw/lists

## Make the Soup for the first page

In [14]:
page = requests.get(base_url)
soup = BeautifulSoup(page.text, 'html.parser')

### Write the Soup to a file for future analysis

In [27]:
with open('lexile/raw/lists/page=0.html', 'wb') as f:
    f.write(soup.encode('utf-8'))

## Make the Soup for every page and save it for future analysis

In [34]:
for p in all_pages_list:
    page = requests.get(p)
    soup = BeautifulSoup(page.text, 'html.parser')
    with open("lexile/raw/lists/" + p.split("?")[1] + ".html", 'wb') as f:
        f.write(soup.encode('utf-8'))

# Scrape the covers
## Make a folder to store the covers

In [35]:
!mkdir lexile/covers

## Make the soup for the first page

In [36]:
page = requests.get(base_url)
soup = BeautifulSoup(page.text, 'html.parser')

In [47]:
covers = soup.findAll(class_="field-content review-product-image")
covers = [cover.findAll("img") for cover in covers]
title = [cover[1].get('title') for cover in covers]
title = [t[:-18].replace("/", "_") for t in title]
cover_src = [cover[1].get('src') for cover in covers]
info = dict(zip(title, cover_src))

In [53]:
for k, v in info.items():
    if '.jpg?' in v:
        try:
            with open('./lexile/covers/' + k + '.jpg', 'wb') as f:
                f.write(requests.get(v).content)
        except FileNotFoundError as err:
            with open('lexile/errors.txt', 'a') as f:
                f.write(k)
    else:
        try:
            with open('./lexile/covers/' + k + '.png', 'wb') as f:
                f.write(requests.get(v).content)
        except FileNotFoundError as err:
            with open('lexile/errors.txt', 'a') as f:
                f.write(k)

## Make the soup for the rest of the pages

In [56]:
for page in all_pages_list: 
    
    page = requests.get(page)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    covers = soup.findAll(class_="field-content review-product-image")
    covers = [cover.findAll("img") for cover in covers]
    
    title = [cover[1].get('title') for cover in covers]
    title = [t[:-18].replace("/", "_") for t in title]
    
    cover_src = [cover[1].get('src') for cover in covers]
    
    info = dict(zip(title, cover_src))
    
    for k, v in info.items():
        if '.jpg?' in v:
            try:
                with open('./lexile/covers/' + k + '.jpg', 'wb') as f:
                    f.write(requests.get(v).content)
            except FileNotFoundError as err:
                with open('lexile/errors.txt', 'a') as f:
                    f.write(k)
        
        else:
            try:
                with open('./lexile/covers/' + k + '.png', 'wb') as f:
                    f.write(requests.get(v).content)
            except FileNotFoundError as err:
                with open('lexile/errors.txt', 'a') as f:
                    f.write(k)

In [57]:
change = []
for k, v in info.items():
    if '.jpeg?' in v:
        change.append(k)

In [59]:
!mkdir lexile/jpeg

In [63]:
for page in all_pages_list: 
    
    page = requests.get(page)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    covers = soup.findAll(class_="field-content review-product-image")
    covers = [cover.findAll("img") for cover in covers]
    
    title = [cover[1].get('title') for cover in covers]
    title = [t[:-18].replace("/", "_") for t in title]
    
    cover_src = [cover[1].get('src') for cover in covers]
    
    info = dict(zip(title, cover_src))
    
    with open('cover_info.txt', 'a') as json_file:
      json.dump(info, json_file)
    
    for k, v in info.items():
        if '.jpg?' in v:
            continue
        elif '.png?' in v: 
            continue 
        else:
            try:
                with open('./lexile/jpeg/' + k + '.jpeg', 'wb') as f:
                    f.write(requests.get(v).content)
            except FileNotFoundError as err:
                with open('lexile/errors.txt', 'a') as f:
                    f.write(k)

# To do: 
- ~~make a list of all individual book pages~~
- scrape each page
- save the results in their own files
- merge jpeg with the covers folder
- refactor code